# Overview

Data Source:
- Use data from the [Chicago Data Portal](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2): Crimes 2001 to Present, which includes type of crime, date/time, lat/long, District/ward, arrests, etc.

Helper Notebook:
- Use the helper notebook in this [repository](https://github.com/coding-dojo-data-science/preparing-chicago-crime-data) to process your manually-downloaded csv into several .csv.gz files

Supplemental Data:
- To answer some of the possible questions, you may need to perform some feature engineering, like adding holiday information from an API on this [Holiday Data](https://docs.google.com/spreadsheets/d/1d8hoZzDAhbWx6EwNjrMTTOE5-23Pr1VxJeUxVj1JL9U/edit?usp=sharing).

## Possible Questions to consider:
1) Comparing Police Districts:
 - Which district has the most crimes? Which has the least?
2) Crimes Across the Years:
  - Are the total number of crimes increasing or decreasing across the years?
  - Are there any individual crimes that are doing the opposite (e.g decreasing when overall crime is increasing or vice-versa)?
3) Comparing AM vs. PM Rush Hour:
  - Are crimes more common during AM rush hour or PM rush hour?
    - You can consider any crime that occurred between 7 AM - 10 AM as AM rush hour
    - You can consider any crime that occurred between 4 -7 PM as PM rush hour.
  - What are the top 5 most common crimes during AM rush hour? What are the top 5 most common crimes during PM rush hour?
  - Are Motor Vehicle Thefts more common during AM rush hour or PM Rush Hour?
4) Comparing Months:
  - What months have the most crime? What months have the least?
  - Are there any individual crimes that do not follow this pattern? If so, which crimes?
5) Comparing Holidays:
  - Are there any holidays that show an increase in the # of crimes?
  - Are there any holidays that show a decrease in the # of crimes?

# Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mticks
import datetime as dt
from matplotlib import dates as mdates

import warnings
warnings.filterwarnings('ignore')

# Functions

In [2]:
# import data into variables
def import_files():
    """Reads in .csv files from Data folder and names them
    dynamically with appropriate year"""
    
    
    file_name_template = "Chicago-Crime_{}"
    years = range(2001, 2024)
    
    # initialize dictionary to save files to
    data = {}
    
    # iterate through years
    for year in years:
        
        # recreate the file name
        file_name = file_name_template.format(year)
        
        # save file path based on file name
        file_path = f"Data/{file_name}.csv"
        
        # import as df
        df = pd.read_csv(file_path)
        
        # store in dictionary
        data[file_name] = df
        
        # replace dash (not allowed in python variable
        # names) with underscore
        # initialize new dictionary
        new_data = {}
            
        # loop through dict items
        for key, value in data.items():
            
            # if dash in key
            if "-" in key:
                
                # replace dash with underscore and save as new key
                new_key = key.replace("-", "_")
                
            # else new key is same as old key
            else:
                new_key = key
                
            # add new key with corresponding value to new dict
            new_data[new_key] = value
        
    # return dictionary
    return new_data

In [3]:
# function to format y-axis units
def thousands(x, pos):
    """formats count in thousands"""
    new_x = x / 1000
    return f"{new_x:,.0f}K"

# Data Loading

In [4]:
# call function and store in variable
imported_data = import_files()

# assign values to variables based on key names
for key, value in imported_data.items():
    globals()[key] = value
    
Chicago_Crime_2023

,ID,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Latitude,Longitude
0,12939189,01/01/2023 01:00:00 AM,OTHER OFFENSE,OTHER VEHICLE OFFENSE,STREET,False,True,423,4.0,7.0,41.736726,-87.556955
1,12944345,01/01/2023 01:00:00 AM,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,633,6.0,9.0,41.728305,-87.613136
2,12938688,01/01/2023 01:00:00 AM,MOTOR VEHICLE THEFT,THEFT / RECOVERY - AUTOMOBILE,STREET,False,False,1632,16.0,38.0,41.944491,-87.787524
3,12944392,01/01/2023 01:00:00 PM,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,915,9.0,11.0,41.833927,-87.641312
4,12943227,01/01/2023 01:00:00 AM,ROBBERY,STRONG ARM - NO WEAPON,CTA TRAIN,False,False,1132,11.0,24.0,41.873907,-87.725430
...,...,...,...,...,...,...,...,...,...,...,...,...
122001,13122057,06/28/2023 12:47:00 AM,ROBBERY,ARMED - HANDGUN,CAR WASH,False,False,1221,12.0,36.0,41.890695,-87.684644
122002,13122505,06/28/2023 12:50:00 PM,DECEPTIVE PRACTICE,ATTEMPT - FINANCIAL IDENTITY THEFT,APARTMENT,False,False,1931,19.0,32.0,41.934429,-87.675762
122003,13122064,06/28/2023 12:50:00 AM,ROBBERY,ARMED - HANDGUN,STREET,False,False,1135,11.0,28.0,41.867514,-87.686846
122004,13122624,06/28/2023 12:50:00 PM,ROBBERY,AGGRAVATED,SIDEWALK,False,False,513,5.0,9.0,41.694309,-87.620794


# Data Prep

In [5]:
df_list = [globals()[name] for name in imported_data.keys() if isinstance(globals()[name], pd.DataFrame)]
df = pd.concat(df_list)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7834343 entries, 0 to 122005
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ID                    int64  
 1   Date                  object 
 2   Primary Type          object 
 3   Description           object 
 4   Location Description  object 
 5   Arrest                bool   
 6   Domestic              bool   
 7   Beat                  int64  
 8   District              float64
 9   Ward                  float64
 10  Latitude              float64
 11  Longitude             float64
dtypes: bool(2), float64(4), int64(2), object(4)
memory usage: 672.4+ MB


In [6]:
df

,ID,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Latitude,Longitude
0,1323184,01/01/2001 01:00:00 PM,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,False,2213,22.0,NaN,41.707671,-87.666996
1,1328315,01/01/2001 01:00:00 AM,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,RESIDENCE,False,False,725,7.0,NaN,41.771269,-87.662929
2,1311933,01/01/2001 01:00:00 AM,SEX OFFENSE,CRIMINAL SEXUAL ABUSE,RESIDENCE,True,False,1434,14.0,NaN,41.910797,-87.682214
3,1330412,01/01/2001 01:00:00 AM,THEFT,$500 AND UNDER,TAVERN/LIQUOR STORE,False,False,1813,18.0,NaN,41.917383,-87.648623
4,1311735,01/01/2001 01:00:00 AM,BATTERY,AGGRAVATED: OTHER DANG WEAPON,STREET,False,False,1632,16.0,NaN,41.938196,-87.800534
...,...,...,...,...,...,...,...,...,...,...,...,...
122001,13122057,06/28/2023 12:47:00 AM,ROBBERY,ARMED - HANDGUN,CAR WASH,False,False,1221,12.0,36.0,41.890695,-87.684644
122002,13122505,06/28/2023 12:50:00 PM,DECEPTIVE PRACTICE,ATTEMPT - FINANCIAL IDENTITY THEFT,APARTMENT,False,False,1931,19.0,32.0,41.934429,-87.675762
122003,13122064,06/28/2023 12:50:00 AM,ROBBERY,ARMED - HANDGUN,STREET,False,False,1135,11.0,28.0,41.867514,-87.686846
122004,13122624,06/28/2023 12:50:00 PM,ROBBERY,AGGRAVATED,SIDEWALK,False,False,513,5.0,9.0,41.694309,-87.620794


In [7]:
# Null Values
df.isna().sum()

ID                           0
Date                         0
Primary Type                 0
Description                  0
Location Description     10679
Arrest                       0
Domestic                     0
Beat                         0
District                    47
Ward                    614849
Latitude                 87853
Longitude                87853
dtype: int64

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format = '%m/%d/%Y %I:%M:%S %p')

df['Date']

In [ ]:
# set datetime column as index
chicago_crime = df.set_index('Date')

chicago_crime

In [ ]:
chicago_resampled_df = chicago_crime.copy()
chicago_resampled_df['Total Crimes'] = 1

# resample and count with .sum()
chicago_resampled_df = chicago_resampled_df.resample('D').count()

chicago_resampled_df.head()

In [ ]:
chicago_resampled_df = chicago_crime.copy()
chicago_resampled_df['Total Crimes'] = 1

# resample and count with .sum()
chicago_resampled_df = chicago_resampled_df.resample('D').sum(numeric_only = True)

chicago_resampled_df.head()

I now have my data ready:
- **chicago_crime** has one row per instance of a crime

# Questions

## 1. Which district has the most crimes and which has the least?

In [ ]:
chicago_crime['District'] = chicago_crime['District'].astype('Int64')
chicago_crime.info()

In [ ]:
# counts per district
chicago_crime['District'].value_counts()

In [ ]:
# percentages of crimes
chicago_crime['District'].value_counts(normalize = True)

### Visualization

In [ ]:
district_info = chicago_crime['District'].value_counts()

fig, ax = plt.subplots(figsize = (10, 8))
ax = sns.barplot(x = district_info.index, y = district_info.values)


fmt_thous = mticks.FuncFormatter(thousands)
ax.yaxis.set_major_formatter(fmt_thous)

ax.set_title('Total Crimes by District(2001-2022)', fontsize = 15, weight = 'bold')
ax.set_ylabel('Number of Crimes (in Thousands)', fontsize = 15, weight = 'bold')
ax.set_xlabel('District', fontsize = 15, weight = 'bold');

In [ ]:
district_info = chicago_crime['District'].value_counts(dropna = True, normalize = True)

fig, ax = plt.subplots(figsize = (10, 8))
ax = sns.barplot(x = district_info.index, y = district_info.values)

ax.set_title('Percentage of Crimes by District(2001-2022)', fontsize = 20, weight = 'bold')
ax.set_ylabel('Percentage of Crimes', fontsize = 15, weight = 'bold')
ax.set_xlabel('District', fontsize = 15, weight = 'bold');

### Analysis

Which district has the most crimes? Which has the least?
- **District 8** has the **most** crimes with **526293 total crimes**. This represents **6.72%** of the total crimes reported.
- **District 21** has the **least** crimes with **4** total crimes. This represents **0.0001%** of the total crimes reported.

## 2a. Are the total number of yearly crimes increasing or decreasing? 

In [ ]:
yearly_crime = chicago_crime.resample('Y').size()

yearly_crime

### Visualization

In [ ]:
fig, ax = plt.subplots(figsize = (10, 8))
ax = sns.lineplot(x = yearly_crime.index, y = yearly_crime.values)

fmt_thous = mticks.FuncFormatter(thousands)
ax.yaxis.set_major_formatter(fmt_thous)

major_ticks_loc = mdates.YearLocator(1)
major_ticks_format = mdates.DateFormatter('%Y')
x_years = pd.date_range(start = '2002', end = '2024', freq = 'AS')
plt.xticks(x_years, [year.year for year in x_years])

plt.xticks(rotation = 60)
ax.grid()

ax.set_title('Total Crimes by Year(2001-2022)', fontsize = 20, weight = 'bold')
ax.set_ylabel('Total Crimes', fontsize = 15, weight = 'bold')
ax.set_xlabel('Year', fontsize = 15, weight = 'bold');

### Analysis

Are the total number of yearly crimes increasing or decreasing? 
- Crime rate over the years has been **Decreasing**

## 2b. Are there any individual crimes that are trending the opposite way? 

In [ ]:
chicago_crime['Primary Type'].value_counts()

- I need to clean up the column headers `CRIM SEXUAL ASSAULT`,`OTHER NARCOTIC VIOLATION`, `NON - CRIMINAL`, and `NON-CRIMINAL (SUBJECT SPECIFIED)`.

In [ ]:
chicago_crime['Primary Type'] = chicago_crime['Primary Type'].replace(
    {'CRIM SEXUAL ASSAULT': 'CRIMINAL SEXUAL ASSAULT',                                                 
     'NON - CRIMINAL': 'NON-CRIMINAL',                                                     
     'NON-CRIMINAL (SUBJECT SPECIFIED)': 'NON-CRIMINAL',                                                 
     'OTHER NARCOTIC VIOLATION': 'NARCOTICS'})

chicago_crime['Primary Type'].value_counts()

In [ ]:
specific_crimes = chicago_crime.copy()
specific_crimes['Count'] = 1

specific_crimes.info()

In [ ]:
specific_crimes = specific_crimes.groupby('Primary Type').resample('Y').count()

drop_these = ['ID', 'Arrest', 'Domestic', 'Beat', 'District', 'Ward', 'Latitude',
              'Longitude', 'Primary Type', 'Description', 'Location Description']
specific_crimes.drop(columns = drop_these, inplace = True)

specific_crimes

In [ ]:
crime_unstacked = specific_crimes.unstack(level = 0)

crime_unstacked

### Visualization

In [ ]:
for i, column in enumerate(crime_unstacked.columns):
    fig, ax = plt.subplots(figsize = (10, 8))
    crime_unstacked[column].plot(ax = ax)

    fmt_thous = mticks.FuncFormatter(thousands)
    ax.yaxis.set_major_formatter(fmt_thous)

    major_ticks_loc = mdates.YearLocator(1)
    major_ticks_format = mdates.DateFormatter('%Y')
    x_years = pd.date_range(start = '2002', end = '2024', freq = 'AS')
    plt.xticks(x_years, [year.year for year in x_years])

    plt.xticks(rotation = 60)
    ax.grid()

    ax.set_title(f'Total {column[1].title()} Crimes by Year(2001-2022)', fontsize = 20, weight = 'bold')
    ax.set_ylabel('Total Crimes', fontsize = 15, weight = 'bold')
    ax.set_xlabel('Year', fontsize = 15, weight = 'bold');

### Analysis

- The following crime categories are trending different from the overall crime statistics:
  - CONCEALED CARRY LICENSE VIOLATION: Consistently low until 2013 and then peaked in 2019. Has been consistently high the last few years
  - CRIMINAL SEXUAL ASSAULT: Consistent throughout the years
  - DECEPTIVE PRACTICE: Consistent until 2014 then peaked in 2018 but now on the decline
  - DOMESTIC VIOLENCE: Basically nonexistent. Not enough into to determine a trend
  - HOMICIDE: Consistent until 2015 then had a huge increase in 2016. Decreased in 2017, but now is at its peak in 2022
  - HUMAN TRAFFICKING: Consistently on the rise since 2012
  - INTERFERENCE WITH PUBLIC OFFICER: Increased consistently until 2019, then had a sharp decline
  - MOTOR VEHICLE THEFT: Following the trend and decreasing until 2021 but had an uptick in 2022
  - NON-CRIMINAL: Consistent until 2011 then increased until its peak in 2016. Now back down to pre-2011 levels
  - OBSCENITY: Consistent until 2014 then had a increase until its peak in 2017 and 2018 now it is on the decline
  - PUBLIC INDECENCY: Overall low but had a swell from 2011 - 2020
  - PUBLIC PEACE VIOLATION: Had an increase from 2001 - 2010, but has been consistently decreasing since then
  - RITUALISM: Had a spike in 2006 but overall has been very low
  - STALKING: Consistently low until about 2020 then more than doubles in 2021
  - WEAPONS VIOLATION: consistent from  from 2001 to 2016, then had a very steep increase since then. More than double annually

## 3a. Which months have the most crime? Which have the least?

In [ ]:
monthly = df.copy()
monthly['Month'] = monthly['Date'].dt.month_name()

monthly['Month'].nunique()

In [ ]:
monthly['Month'].unique()

In [ ]:
months = monthly['Month'].value_counts(dropna = False)

months

### Visualization

In [ ]:
fig, ax = plt.subplots(figsize = (10, 8))
ax = sns.barplot(x = months.index, y = months.values)


fmt_thous = mticks.FuncFormatter(thousands)
ax.yaxis.set_major_formatter(fmt_thous)

# set title and axes
ax.set_title('Total Crimes by Month(2001-2022) \n Sorted by Total Crime', fontsize = 20, weight = 'bold')
ax.set_ylabel('Number of Crimes (in Thousands)', fontsize = 20, weight = 'bold')
ax.set_xlabel('Month', fontsize = 15, weight = 'bold')

# rotate x-ticks
plt.xticks(rotation = 90);

In [ ]:
month_order = ['January','February','March','April','May','June','July',
               'August','September','October','November','December']

months_in_order = months.reindex(month_order)

months_in_order

In [ ]:
fig, ax = plt.subplots(figsize = (10, 8))
ax = sns.barplot(x = months_in_order.index, y = months_in_order.values)


fmt_thous = mticks.FuncFormatter(thousands)
ax.yaxis.set_major_formatter(fmt_thous)

# set title and axes
ax.set_title('Total Crimes by Month(2001-2022) \n Sorted by Month Name', fontsize = 20, weight = 'bold')
ax.set_ylabel('Number of Crimes (in Thousands)', fontsize = 20, weight = 'bold')
ax.set_xlabel('Month', fontsize = 15, weight = 'bold')

# rotate x-ticks
plt.xticks(rotation = 90);

### Analysis

Which months have the most crime? Which have the least?
- **July** has the **most** crimes committed with **717168** total crimes. **August** has the second highest crimes committed with **710369** total crimes.

- **February** has the **least** crimes committed with **547594** total crimes. **December** has the second lowest crimes committed with **579679** total crimes.     

## 3b. Are there any individual crimes that do not follow this pattern?

In [ ]:
monthly['Primary Type'].value_counts()

In [ ]:
monthly['Primary Type'] = monthly['Primary Type'].replace(
    {'CRIM SEXUAL ASSAULT': 'CRIMINAL SEXUAL ASSAULT',                                                 
     'NON - CRIMINAL': 'NON-CRIMINAL',                                                     
     'NON-CRIMINAL (SUBJECT SPECIFIED)': 'NON-CRIMINAL',                                                 
     'OTHER NARCOTIC VIOLATION': 'NARCOTICS'})

monthly['Primary Type'].value_counts()

In [ ]:
monthly_crime = monthly.groupby(['Primary Type', 'Month']).size()

monthly_crime

In [ ]:
unstacked_monthly_crime = monthly_crime.unstack(level = 0)

unstacked_monthly_crime

In [ ]:
unstacked_monthly_crime = unstacked_monthly_crime.astype('Int64')

unstacked_monthly_crime.isna().sum()

In [ ]:
unstacked_monthly_crime.fillna(value = 0, inplace = True)
unstacked_monthly_crime.isna().sum()

### Visualization

In [ ]:
fig, axes = plt.subplots(len(unstacked_monthly_crime.columns),
                        figsize = (10, 8 * len(unstacked_monthly_crime.columns)))  
    

for i, column in enumerate(unstacked_monthly_crime):
  
    ax = axes[i]

    fmt_thous = mticks.FuncFormatter(thousands)
    ax.yaxis.set_major_formatter(fmt_thous)


    month_datetime = pd.to_datetime(unstacked_monthly_crime.index, format = '%B').sort_values()
    x_months = range(len(unstacked_monthly_crime))
    x_labels = [m.strftime('%B') for m in month_datetime]
    ax.set_xticks(x_months)
    ax.set_xticklabels(x_labels, rotation = 60)
    

    ax.set_title(f'Total {column.title()} Crimes by Month', fontsize = 20, weight = 'bold')
    ax.set_ylabel('Total Crimes', fontsize = 15, weight = 'bold')
    ax.set_xlabel('Month', fontsize = 15, weight = 'bold')


    ax.bar(x_months, unstacked_monthly_crime[column])
    ax.tick_params(axis = 'x', rotation = 60)

plt.tight_layout()

### Analysis

- These crimes follow the trends that have the most offenses in warmer months(June - September) and the least in the winter(December - March):
  - ASSAULT: Most in fall (September) and the least in spring (March)
  - CRIMINAL SEXUAL ASSAULT: Most in summer (June) and the least in spring (March)
  - CRIMINAL TRESPASS: Most in summer (August) and the least in spring (March)
  - LIQUOR LAW VIOLATION: Most in summer (July) and the least in spring (March)
  - NARCOTICS: Most in summer (August) and the least in spring (March)  
  - OTHER OFFENSE: Most in summer (August) and the least in spring (March)
  - SEX OFFENSE: Most in summer (June) and the least in spring (March)
  - STALKING: Most in summer (July) and the least in spring (March)



- These crimes follow a different trend from the overall trend:
  - ARSON:  Most in summer (June) and the least in spring (April)
  - BATTERY: Most in fall (September) and the least in spring (April)
  - BURGLARY: Most in winter (February) and the least in spring (April)
  - CONCEALED CARRY LICENSE VIOLATION: Most in winter (December) and the least in spring (March)
  - CRIMINAL DAMAGE: Most in summer (June) and the least in spring (April)
  - DECEPTIVE PRACTICE: Most in spring (May) and the least in spring (April)
  - DOMESTIC VIOLENCE: The only instances tracked were in spring (May)
  - GAMBLING: Most in winter (February) and the least in spring (March)
  - HOMICIDE: Most in summer (June) and the least in spring (April)
  - HUMAN TRAFFICKING: Most in spring (May) and the least in fall (October)
  - INTERFERENCE WITH PUBLIC OFFICER: Most in winter (February) and the least in spring (March)
  - INTIMIDATION: Most in fall (November) and the least in fall (October)
  - KIDNAPPING: Most in fall (September) and the least in spring (April)
  - MOTOR VEHICLE THEFT: Most in fall (November) and the least in spring (April)
  - NON-CRIMINAL: Most in summer (July) and the least in summer (August)
  - OBSCENITY: Most in fall (November) and the least in spring (February)
  - OFFENSE INVOLVING CHILDREN: Most in Spring (May) and the least in fall (October)
  - PROSTITUTION: Most in spring (May) and the least in spring (March)
  - PUBLIC INDECENCY: Most in winter (February) and the least in spring (May)
  - PUBLIC PEACE VIOLATION: Most in fall (September) and the least in spring (March)
  - RITUALISM: Most in several months (February, March, May, August, November, December and the least in spring (July)
  - ROBBERY: Most in fall (November) and the least in spring (April)
  - THEFT: Most in winter (February) and the least in spring (April)
  - WEAPONS VIOLATION: Most in fall (September) and the least in spring (April)

- Although several crimes follow a slightly different trend, the vast majority of them increase during the warmer months and decrease during the colder ones. 